# Project: Youtube Analyzer
## Authors: Karmehr Arora, Alfred Koh

### Project Overview
Data: Youtube API’s
GUI: Tkinter

Roles:

    Karmehr: Accessing Data from the API & organizing it

    Alfred: Analyzing the data & representing it with tkinter

Goal: 
Finding the top 10 trending basketball videos (or videos of a requested topic) in the last 24 hours
Look for the most (total likes + total comments + total shares) under tag = basketball in the last 24 hours

Limitations: 
Estimated limit of searches of 50 per day due to Youtube API usage restrictions


In [114]:
# import utilized libraries pertaining to data collection
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

In [115]:
# get access key to API
# don't push to github, but hardcode for demo
from getpass import getpass
API_KEY = getpass('Enter your API Key:')

Enter your API Key:········


In [116]:
# define the api service and version
api_service_name = 'youtube'
api_version = 'v3'

## Requesting User to choose topic
default to basketball as topic if user has error/opts out of entering topic

In [117]:
# request user for topic (default = 'basketball')
topic = 'basketball'
topic = input()

## Search youtube for videos with that tag

In [140]:
# access the youtube api client
youtube = googleapiclient.discovery.build(api_service_name, api_version, developerKey = API_KEY)

# make a request to the api to find videos under that topic
# note: cost of this operation is 100 units of the alloted 10,000 units per 24 hours
request = youtube.search().list(
        part="id,snippet",
        maxResults=50,
        q=topic
    )

In [141]:
# execute the query in the request
response = request.execute()

In [188]:
#   for(var i in results.items) {
#     var item = results.items[i];
#     Logger.log('[%s] Title: %s', item.id.videoId, item.snippet.title);
#   }
print("Search ID: ", response["items"][0]['id'])
print("\nSnippet Data:",response["items"][0]['snippet'])
videoIDs = []
videos = response["items"]
count = 1
for video in videos:
    if('videoId' in video['id']):
        print(count, ": https://www.youtube.com/watch?v=" + video['id']['videoId'])
        videoIDs.append(video['id']['videoId'])
        count = count+1
    else: 
        # this seems to only print out channels meaning the youtube search only returns videos & channels
        print(video['snippet'])

Search ID:  {'kind': 'youtube#video', 'videoId': 'A-Ja3fXfpuM'}

Snippet Data: {'publishedAt': '2023-11-05T03:38:45Z', 'channelId': 'UC52N5l5eQWtk05YMJNdIAfg', 'title': 'Summer Nostalgia - Dua Lipa, Avicii, David Guetta, Kygo, Calvin Harris, Alesso, One Republic, Flume', 'description': 'Summer Nostalgia - Dua Lipa, Avicii, David Guetta, Kygo, Calvin Harris, Alesso, One Republic, Flume Summer Nostalgia - Dua ...', 'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/A-Ja3fXfpuM/default_live.jpg', 'width': 120, 'height': 90}, 'medium': {'url': 'https://i.ytimg.com/vi/A-Ja3fXfpuM/mqdefault_live.jpg', 'width': 320, 'height': 180}, 'high': {'url': 'https://i.ytimg.com/vi/A-Ja3fXfpuM/hqdefault_live.jpg', 'width': 480, 'height': 360}}, 'channelTitle': 'Helios Radio', 'liveBroadcastContent': 'live', 'publishTime': '2023-11-05T03:38:45Z'}
1 : https://www.youtube.com/watch?v=A-Ja3fXfpuM
2 : https://www.youtube.com/watch?v=g_CEifuFfrs
3 : https://www.youtube.com/watch?v=WCQoEb_ZYKk
4 : https

## Calculate and Find top 10 most trending videos under that tag

In [136]:
collection_of_video_data = {}
for video in videos:
    if('videoId' in video['id']):
        request = youtube.videos().list(part = 'snippet, statistics', id = video['id']['videoId'])
        response = request.execute()
        collection_of_video_data[video['id']['videoId']] = response

In [163]:
set_of_trend_totals = []
count = 1
for vid_data in collection_of_video_data:
#     print(count, ':', collection_of_video_data[vid_data])
    print("Statistics: ", collection_of_video_data[vid_data]['items'][0]['statistics'])
    
    trendTotal = 0
    if "viewCount" in collection_of_video_data[vid_data]['items'][0]['statistics']:
        views = collection_of_video_data[vid_data]['items'][0]['statistics']['viewCount']
        trendTotal = trendTotal + int(views)
    if "likeCount" in collection_of_video_data[vid_data]['items'][0]['statistics']:
        likes = collection_of_video_data[vid_data]['items'][0]['statistics']['likeCount']
        trendTotal = trendTotal + int(likes)
    if "commentCount" in collection_of_video_data[vid_data]['items'][0]['statistics']:
        comments = collection_of_video_data[vid_data]['items'][0]['statistics']['commentCount']
        trendTotal = trendTotal + int(comments)

    set_of_trend_totals.append(trendTotal)
    print(count, ': Trend Total: ', trendTotal)
    count = count+1

Statistics:  {'viewCount': '114755', 'likeCount': '258', 'favoriteCount': '0', 'commentCount': '0'}
1 : Trend Total:  115013
Statistics:  {'viewCount': '3604744', 'likeCount': '17289', 'favoriteCount': '0', 'commentCount': '0'}
2 : Trend Total:  3622033
Statistics:  {'viewCount': '453631', 'likeCount': '1994', 'favoriteCount': '0', 'commentCount': '0'}
3 : Trend Total:  455625
Statistics:  {'likeCount': '3', 'favoriteCount': '0', 'commentCount': '0'}
4 : Trend Total:  3
Statistics:  {'viewCount': '263323', 'likeCount': '8', 'favoriteCount': '0', 'commentCount': '0'}
5 : Trend Total:  263331
Statistics:  {'viewCount': '1036723', 'likeCount': '5', 'favoriteCount': '0', 'commentCount': '0'}
6 : Trend Total:  1036728
Statistics:  {'viewCount': '285142', 'likeCount': '43', 'favoriteCount': '0', 'commentCount': '0'}
7 : Trend Total:  285185
Statistics:  {'viewCount': '323934', 'likeCount': '5171', 'favoriteCount': '0', 'commentCount': '436'}
8 : Trend Total:  329541
Statistics:  {'viewCount'

In [186]:
count = 0
video_id_to_trend_total = {}
for video in videoIDs:
    print('videoID: trendTotal --> ', video, end = ": ")
    print(set_of_trend_totals[count])
    video_id_to_trend_total[video] = set_of_trend_totals[count]
    count = count + 1
    
# for video in video_id_to_trend_total:
#     print(video, ":", video_id_to_trend_total[video])

videoID: trendTotal -->  A-Ja3fXfpuM: 115013
videoID: trendTotal -->  g_CEifuFfrs: 3622033
videoID: trendTotal -->  WCQoEb_ZYKk: 455625
videoID: trendTotal -->  rHRK569gMIU: 3
videoID: trendTotal -->  d_iv5RIDBF8: 263331
videoID: trendTotal -->  cA7CnAiXdJk: 1036728
videoID: trendTotal -->  vw_5rIXWKqQ: 285185
videoID: trendTotal -->  U2XMNXtosZQ: 329541
videoID: trendTotal -->  5fUwu9XnmN8: 1019507
videoID: trendTotal -->  7VaVc-PTuAY: 225339
videoID: trendTotal -->  Bmus9_a_hSI: 2357877
videoID: trendTotal -->  QoY84flTdAY: 823248
videoID: trendTotal -->  xyDKZFaqRYg: 314880
videoID: trendTotal -->  ohXLl2rN498: 192686
videoID: trendTotal -->  YXLzJfipmes: 784836
videoID: trendTotal -->  gDKr0ImyWVY: 350469
videoID: trendTotal -->  Q1giqvTwZqs: 261623
videoID: trendTotal -->  SujS6jpa5ug: 607111
videoID: trendTotal -->  7SvDjLdwyRE: 289396
videoID: trendTotal -->  n1AqQT2oLbo: 216674
videoID: trendTotal -->  FWRD0_Nn724: 274055
videoID: trendTotal -->  KUtN89TXLGg: 42077099
videoID: 